# 🛠️ Week 07-08 · Notebook 12 · Building Agents with ReAct & Tools

Construct a LangChain agent that reasons step-by-step, invokes manufacturing tools, and maintains governance logs.

## 🎯 Learning Objectives
- Implement a ReAct-style agent using LangChain `initialize_agent`.
- Register manufacturing-specific tools (spare parts API, downtime reporter).
- Enforce tool call policies and rate limits.
- Capture reasoning trace for compliance review.

## 🧩 Scenario
Technician asks: "Press 14 failing vibration test. Do we have bearings in stock and what are first steps?" Agent should reason, check inventory, and respond with SOP references.

In [ ]:
from langchain.agents import Tool, initialize_agent, AgentType
from langchain_openai import ChatOpenAI
import json
from typing import Dict, Any, List, Optional

# --- Tool Definitions ---
# These functions simulate calls to external APIs or databases.

def spare_parts_lookup(query: str) -> str:
    """
    Looks up spare parts inventory. Query can be a part number, description, or equipment type.
    Returns inventory status and location.
    """
    # In a real implementation, this would query an inventory API.
    inventory = {
        "bearing": {
            "SKF-22210E": {"stock": 12, "location": "Warehouse B-7-3", "compatible": ["Press 12", "Press 14"]},
            "FAG-NU1010-M1": {"stock": 3, "location": "Warehouse B-2-5", "compatible": ["Press 8", "Press 14"]},
        },
        "pressure_sensor": {
            "Siemens-PS1034": {"stock": 5, "location": "Maintenance Office", "compatible": ["Press 10", "Press 14"]}
        }
    }
    query_lower = query.lower()
    results = [f"{part_id}: {details['stock']} in stock" for category, parts in inventory.items() if category in query_lower for part_id, details in parts.items()]
    return f"Found {len(results)} matching parts:\n" + "\n".join(results) if results else "No matching parts found."

def downtime_report(equipment: str) -> str:
    """
    Retrieves maintenance history for specific equipment.
    Returns last 3 maintenance events.
    """
    maintenance_records = {
        "press 14": [
            {"date": "2025-09-30", "issue": "Vibration exceeding threshold", "resolution": "Bearing replacement"},
            {"date": "2025-07-15", "issue": "Hydraulic pressure drop", "resolution": "Seal replacement"},
        ]
    }
    records = maintenance_records.get(equipment.lower(), [])
    report = f"Downtime report for {equipment.upper()}:\n" + "\n".join([f"- {r['date']}: {r['issue']} - {r['resolution']}" for r in records])
    return report if records else f"No maintenance records found for {equipment}."

def sop_lookup(procedure_type: str) -> str:
    """
    Retrieves Standard Operating Procedures for maintenance tasks.
    """
    sops = {
        "bearing replacement": "SOP-122: 1. SAFETY: Perform lockout/tagout. 2. Remove access panel. 3. Extract old bearing...",
        "vibration test": "SOP-287: 1. Set up analyzer. 2. Run equipment at 50% speed. 3. Record readings..."
    }
    return next((sop for key, sop in sops.items() if key in procedure_type.lower()), "No matching SOP found.")

# --- Agent Setup ---
tools = [
    Tool(name="SparePartsInventory", func=spare_parts_lookup, description="Check inventory for spare parts."),
    Tool(name="MaintenanceHistory", func=downtime_report, description="Get maintenance history for equipment."),
    Tool(name="StandardProcedures", func=sop_lookup, description="Look up standard operating procedures.")
]

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

# --- Run Agent ---
question = "Press 14 is failing its vibration test. Do we have bearings in stock and what are the first steps?"
response = agent.invoke({"input": question})

print("\n\n--- FINAL ANSWER ---")
print(response["output"])

### 🧾 Governance Log
Capture reasoning steps (`intermediate_steps`) and tool outputs with timestamps.

In [ ]:
from datetime import datetime
import hashlib
import json

# --- Governance Logging ---
# Capture the agent's reasoning trace for audit and review.

intermediate_steps = response.get("intermediate_steps", [])

# Format the tool calls and responses for clarity
tool_calls = [{
    "tool": action.tool,
    "input": action.tool_input,
    "output": output
} for action, output in intermediate_steps]

# Create a structured log entry
log_entry = {
    "timestamp": datetime.now().isoformat(),
    "request_id": hashlib.md5(question.encode()).hexdigest()[:8],
    "question": question,
    "tools_used": [call["tool"] for call in tool_calls],
    "reasoning_trace": tool_calls,
    "final_answer": response["output"],
    "governance": {
        "operator_id": "TECH-123",
        "plant": "Pune-Manufacturing-01",
        "contains_safety_info": "SAFETY" in response["output"],
        "contains_sop_reference": "SOP-" in response["output"]
    }
}

# Save the log to a file and print for review
with open("agent_governance_log.jsonl", "a") as f:
    f.write(json.dumps(log_entry) + "\n")

print("\n--- Governance Log Saved ---")
print(json.dumps(log_entry, indent=2))

## 🧪 Lab Assignment
1. **Add EHS Validation Tool**: Create a tool that checks if the agent's response includes required safety disclaimers:
   ```python
   def ehs_validation(response: str) -> str:
       """
       Validates if a maintenance response includes required safety information.
       Returns pass/fail with specific missing items.
       """
       required_safety = [
           {"keyword": "lockout/tagout", "reason": "Equipment must be de-energized"},
           {"keyword": "PPE", "reason": "Personal protective equipment required"},
           {"keyword": "hazard", "reason": "Hazard identification required"},
           {"keyword": "SOP-", "reason": "Standard procedure reference required"}
       ]
       
       missing = []
       for item in required_safety:
           if item["keyword"].lower() not in response.lower():
               missing.append(item)
       
       if missing:
           return f"SAFETY VALIDATION FAILED: Missing {len(missing)} required elements:\n" + \
                  "\n".join([f"- {item['keyword']}: {item['reason']}" for item in missing])
       else:
           return "SAFETY VALIDATION PASSED: All required safety elements present."
           
   # Add to tools list
   tools.append(Tool(
       name="SafetyValidation",
       func=ehs_validation,
       description="Check if response includes all required safety elements"
   ))
   ```

2. **Implement Rate Limiting**: Add a wrapper that enforces maximum tool calls per incident:
   ```python
   class RateLimitedAgent:
       def __init__(self, agent, max_calls=5):
           self.agent = agent
           self.max_calls = max_calls
           self.call_count = 0
       
       def reset(self):
           self.call_count = 0
       
       def invoke(self, input_data):
           def _count_tool_call(tool_name, tool_input):
               self.call_count += 1
               if self.call_count > self.max_calls:
                   raise Exception(f"Rate limit exceeded: {self.call_count}/{self.max_calls} tool calls")
               return self.agent.tools_lookup[tool_name].func(tool_input)
           
           # Replace the agent's tool execution method temporarily
           original_method = self.agent._run_tool
           self.agent._run_tool = _count_tool_call
           
           try:
               result = self.agent.invoke(input_data)
               return result
           finally:
               # Restore original method
               self.agent._run_tool = original_method
               
   # Wrap the agent with rate limiting
   limited_agent = RateLimitedAgent(agent, max_calls=5)
   result = limited_agent.invoke({"input": "Check all maintenance records and parts for presses"})
   ```

3. **Store Agent Logs in MLflow**: Implement MLflow tracking for agent interactions:
   ```python
   import mlflow
   
   def log_agent_run_to_mlflow(question, response, log_entry):
       # Start an MLflow run
       mlflow.start_run(run_name=f"agent_run_{log_entry['request_id']}")
       
       try:
           # Log metrics
           mlflow.log_metric("num_tool_calls", len(log_entry["tool_calls"]))
           mlflow.log_metric("contains_safety_info", 1 if log_entry["governance"]["contains_safety_info"] else 0)
           mlflow.log_metric("contains_sop_reference", 1 if log_entry["governance"]["contains_sop_reference"] else 0)
           
           # Log parameters
           mlflow.log_param("question", question)
           mlflow.log_param("operator_id", log_entry["governance"]["operator_id"])
           mlflow.log_param("plant", log_entry["governance"]["plant"])
           
           # Log artifacts
           with open("agent_response.txt", "w") as f:
               f.write(response["output"])
           mlflow.log_artifact("agent_response.txt")
           
           with open("agent_reasoning.json", "w") as f:
               json.dump(log_entry["tool_calls"], f, indent=2)
           mlflow.log_artifact("agent_reasoning.json")
           
           # Log the complete governance log
           with open("complete_log.json", "w") as f:
               json.dump(log_entry, f, indent=2)
           mlflow.log_artifact("complete_log.json")
           
       finally:
           mlflow.end_run()
   
   # Log the agent run
   log_agent_run_to_mlflow(question, response, log_entry)
   ```

4. **Tabletop Exercise Review Interface**: Create a simple visualization of agent traces for review:
   ```python
   import pandas as pd
   from IPython.display import display, HTML
   
   def create_review_table(log_entry):
       """Create a formatted review table for maintenance leads"""
       steps_df = pd.DataFrame(log_entry["tool_calls"])
       
       # Format the table with styling
       styled_df = steps_df.style.set_table_attributes('class="table table-striped table-bordered"')
       
       # Create HTML summary
       html = f"""
       <h2>Agent Reasoning Review: Request {log_entry['request_id']}</h2>
       <p><strong>Question:</strong> {log_entry['question']}</p>
       <p><strong>Final Answer:</strong> {log_entry['answer']}</p>
       <h3>Reasoning Steps:</h3>
       {styled_df.to_html()}
       <h3>Safety Assessment:</h3>
       <ul>
           <li>Contains Safety Info: {'✅' if log_entry['governance']['contains_safety_info'] else '❌'}</li>
           <li>Contains SOP Reference: {'✅' if log_entry['governance']['contains_sop_reference'] else '❌'}</li>
       </ul>
       <p><em>Reviewer Sign-off: _______________________ Date: _______________________</em></p>
       """
       
       return HTML(html)
   
   # Display the review table
   review_table = create_review_table(log_entry)
   display(review_table)
   
   # Save as HTML file for sharing
   with open("agent_review.html", "w") as f:
       f.write(review_table.data)
   ```

## ✅ Checklist
- [ ] ReAct agent successfully answers manufacturing questions with at least 3 tools
- [ ] Agent reasoning trace fully documented with all tool calls and outputs
- [ ] Governance logs capture execution details, question, answer, and safety compliance
- [ ] EHS validation tool implemented with specific safety requirements
- [ ] Rate limiting wrapper prevents tool call abuse
- [ ] MLflow integration stores all agent runs with proper parameters and artifacts
- [ ] Review interface allows maintenance leads to approve or flag agent decisions

## 📚 References
- LangChain Agents Guide
- ReAct Paper (Yao et al., 2022)
- Week 05 Governance Notebook